In [ ]:
# LIB INSTALLATION

!pip install yfinance
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

In [12]:
# LIB IMPORTS

import json
import os
from datetime import datetime, date

import yfinance as yf

import numpy as np
np.float_ = np.float64

from crewai import Agent, Task

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

In [13]:
# CREATE YAHOO FINANCE TOOL

today = date.today()
last_year = str(datetime.now().year - 1) + "-01-01"

def fetch_stock_price(ticket):
  stock = yf.download(ticket, start=last_year, end=today)

  return stock

yahoo_finance_tool = Tool(
  name = "Yahoo Finance Tool",
  description = "Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
  func = lambda ticket: fetch_stock_price(ticket)
)

In [4]:
# OPENAI LLM - GPT IMPORT

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [8]:
stockPriceAnalyst = Agent(
  role = "Senior stock price Analyst",
  goal = "Find the {ticket} stock price and analyses trends",
  backstory = """You're highly experienced in analyzing the price of a 
  specific stock and make predictions about its future price.""",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  tools = [yahoo_finance_tool],
)

In [9]:
getStockPrice = Task(
  description = "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
  expected_output = """Specify the current trend stock price - up, down or sideways.
  eg. stock = 'APPL, price UP' 
  """,
  agent = stockPriceAnalyst
)

In [ ]:
# SEARCH TOOL IMPORT

search_tool = DuckDuckGoSearchResults(backend = 'news' , num_results = 10)

In [11]:
newsAnalyst = Agent(
  role = "Stock News Analyst",
  goal = """Create a short summary of the market news related to the stock 
  {ticket} company. Specify the current trend - up, down or sideways with the news
  context. For each request stock asset, specify a number between 0 and 100, where 0 is
  extreme fear and 100 is extreme greed.
  """,
  backstory = """You're highly experienced in analyzing the market trends and news and have
  tracked assert for more then 10 years.

  You're also master level analysts in the tradicional markets and have deep understanding
  of human psychology.

  You understand news, theirs titles and information, but you look at those with
  a health dose of skepticism. 
  You consider also the source of the news articles.
  """,
  verbose = True,
  llm = llm,
  max_iter = 10,
  memory = True,
  tools = [search_tool],
)

In [ ]:
get_news = Task(
  description = """Take the stock and always include BTC to it (if not request).
  Use the search tool to search each one individually.

  The current date is {datetime.now()}.

  Compose the results into a helpful report""",
  expected_output = """A summary of the overall market and one sentence summary 
  for each request asset.
  Include a fear/greed score for each asset based on the news. Use format:
  <STOCK ASSET>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED SCORE>
  """,
  agent = newsAnalyst
)